In [6]:
from pandas import read_csv
from pandas import datetime
# from matplotlib import pyplot
from plotly import plotly
import numpy as np

In [7]:
def parser(x):
    return x
    # return datetime.strptime("190"+x, "%Y-%m")

In [8]:
series0 = read_csv("./data/realTweets/Twitter_volume_AAPL.csv", header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
print(series0.head())
series1 = read_csv("./data/realTweets/Twitter_volume_UPS.csv", header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
print(series1.head())

timestamp
2015-02-26 21:42:53    104
2015-02-26 21:47:53    100
2015-02-26 21:52:53     99
2015-02-26 21:57:53    154
2015-02-26 22:02:53    120
Name: value, dtype: int64
timestamp
2015-02-26 21:42:53    2
2015-02-26 21:47:53    2
2015-02-26 21:52:53    4
2015-02-26 21:57:53    3
2015-02-26 22:02:53    5
Name: value, dtype: int64


In [9]:
# # fig = pyplot.figure(1)
# pyplot.plot(series0,label='Actual Values')
# pyplot.plot(series1,label='Forecast Values',linestyle='--')
# pyplot.title('Simple Moving Averages')
# pyplot.grid(True)
# pyplot.legend(loc='upper center',fancybox=True, shadow=True, ncol=2)

# pyplot.show()

In [12]:
data = [dict(
    visible = False,
    line=dict(color='00CED1', width=6),
    name = '𝜈 = '+str(step),
    x = np.arange(0,10,0.01),
    y = np.sin(step*np.arange(0,10,0.01))) for step in np.arange(0,5,0.1)]
data[10]['visible'] = True
steps = []
for i in range(len(data)):
    step = dict(
        method = 'restyle',
        args = ['visible', [False] * len(data)],
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active = 10,
    currentvalue = {"prefix": "Frequency: "},
    pad = {"t": 50},
    steps = steps
)]

layout = dict(sliders=sliders)
fig = dict(data=data, layout=layout)

plotly.iplot(fig, filename='Sine Wave Slider')

ValueError: 
    Invalid value of type 'builtins.str' received for the 'color' property of scatter.line
        Received value: '00CED1'

    The 'color' property is a color and may be specified as:
      - A hex string (e.g. '#ff0000')
      - An rgb/rgba string (e.g. 'rgb(255,0,0)')
      - An hsl/hsla string (e.g. 'hsl(0,100%,50%)')
      - An hsv/hsva string (e.g. 'hsv(0,100%,100%)')
      - A named CSS color:
            aliceblue, antiquewhite, aqua, aquamarine, azure,
            beige, bisque, black, blanchedalmond, blue,
            blueviolet, brown, burlywood, cadetblue,
            chartreuse, chocolate, coral, cornflowerblue,
            cornsilk, crimson, cyan, darkblue, darkcyan,
            darkgoldenrod, darkgray, darkgrey, darkgreen,
            darkkhaki, darkmagenta, darkolivegreen, darkorange,
            darkorchid, darkred, darksalmon, darkseagreen,
            darkslateblue, darkslategray, darkslategrey,
            darkturquoise, darkviolet, deeppink, deepskybl